In [1]:
import sys, os
import pandas as pd

project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from connectors.database import DatabaseConnector
from config.settings import DB_URL

connector = DatabaseConnector(DB_URL)
engine = connector.connect()

In [3]:
from load.to_db import DataBaseLoader
from load.to_file import FileLoader
from extract.from_db import DatabaseExtractor
from extract.from_file import FileExtractor

In [4]:
csv_extractor = FileExtractor("/Users/mohamedmoaaz/Desktop/usecase")
data_frames = csv_extractor.extract_from_csv("source") # dict of dataframes
data_frames.keys()

2025-04-15 21:23:33 => ✅ Extracted 1445 rows from /Users/mohamedmoaaz/Desktop/usecase/source/customers.csv
2025-04-15 21:23:33 => ✅ Extracted 7 rows from /Users/mohamedmoaaz/Desktop/usecase/source/categories.csv
2025-04-15 21:23:33 => ✅ Extracted 334 rows from /Users/mohamedmoaaz/Desktop/usecase/source/products.csv
2025-04-15 21:23:33 => ✅ Extracted 1615 rows from /Users/mohamedmoaaz/Desktop/usecase/source/orders.csv
2025-04-15 21:23:33 => ✅ Extracted 10 rows from /Users/mohamedmoaaz/Desktop/usecase/source/staffs.csv
2025-04-15 21:23:33 => ✅ Extracted 4764 rows from /Users/mohamedmoaaz/Desktop/usecase/source/order_items.csv
2025-04-15 21:23:33 => ✅ Extracted 9 rows from /Users/mohamedmoaaz/Desktop/usecase/source/brands.csv
2025-04-15 21:23:33 => ✅ Extracted 3 rows from /Users/mohamedmoaaz/Desktop/usecase/source/stores.csv
2025-04-15 21:23:33 => ✅ Extracted 939 rows from /Users/mohamedmoaaz/Desktop/usecase/source/stocks.csv


dict_keys(['customers.csv', 'categories.csv', 'products.csv', 'orders.csv', 'staffs.csv', 'order_items.csv', 'brands.csv', 'stores.csv', 'stocks.csv'])

In [ ]:
postgres_loader = DataBaseLoader(engine)
csv_loader = FileLoader("/Users/mohamedmoaaz/Desktop/usecase/datalake")
for file_name in data_frames.keys():
    if file_name == "orders.csv" or file_name == "order_items.csv":
        table_name = file_name.replace(".csv", "")
        postgres_loader.load_to_postgres(data_frames[file_name], table_name)
    else:
        csv_loader.load_to_csv(data_frames[file_name], file_name)

2025-04-15 21:23:34 => ✅ Loaded 1445 rows into customers.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 7 rows into categories.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 334 rows into products.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 1615 rows into orders in PostgreSQL
2025-04-15 21:23:34 => ✅ Loaded 10 rows into staffs.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 4764 rows into order_items in PostgreSQL
2025-04-15 21:23:34 => ✅ Loaded 9 rows into brands.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 3 rows into stores.csv in CSV format
2025-04-15 21:23:34 => ✅ Loaded 939 rows into stocks.csv in CSV format


In [10]:
csv_extractor = FileExtractor("/Users/mohamedmoaaz/Desktop/usecase/")
data_lake_data = csv_extractor.extract_from_csv("datalake") # dict of dataframes
for file_name in data_lake_data.keys():
    data_lake_data[file_name]["source"] = "data lake"

combined_data = pd.concat(data_lake_data.values(), ignore_index=True)

postgres_extractor = DatabaseExtractor(DB_URL)

query = "SELECT * FROM order_items"
db_data = postgres_extractor.extract_from_postgres(query)
db_data["source"] = "postgres db"
combined_data = pd.concat([combined_data, db_data], ignore_index=True)

query = "SELECT * FROM orders"
db_data = postgres_extractor.extract_from_postgres(query)
db_data["source"] = "postgres db"
combined_data = pd.concat([combined_data, db_data], ignore_index=True)

2025-04-15 22:11:25 => ✅ Extracted 1445 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/customers.csv
2025-04-15 22:11:25 => ✅ Extracted 7 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/categories.csv
2025-04-15 22:11:25 => ✅ Extracted 334 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/products.csv
2025-04-15 22:11:25 => ✅ Extracted 10 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/staffs.csv
2025-04-15 22:11:25 => ✅ Extracted 9 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/brands.csv
2025-04-15 22:11:25 => ✅ Extracted 3 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/stores.csv
2025-04-15 22:11:25 => ✅ Extracted 939 rows from /Users/mohamedmoaaz/Desktop/usecase/datalake/stocks.csv
2025-04-15 22:11:25 => ✅ Extracted 4764 rows from PostgreSQL
2025-04-15 22:11:25 => ✅ Extracted 1615 rows from PostgreSQL


In [ ]:
combined_data.to_csv("/Users/mohamedmoaaz/Desktop/usecase/landing/combined_data.csv", index=False)

OSError: Cannot save file into a non-existent directory: '/Users/mohamedmoaaz/Desktop/usecase/landing_zone'